In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# Load datasets
train_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
oil_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
holiday_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
sample_submission = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv')

# Convert 'date' columns to datetime format
train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])
oil_data['date'] = pd.to_datetime(oil_data['date'])
holiday_data['date'] = pd.to_datetime(holiday_data['date'])
#display(train_data) 
#display(test_data)

# Merge the two datasets
merged_data = pd.concat([train_data, test_data], ignore_index=True)

# Display the merged dataset
#print("Merged Dataset:")
#display(merged_data)

# Check for missing values in the oil_data dataset
missing_oil = oil_data['dcoilwtico'].isna().sum()
#print(f"Number of missing values in 'dcoilwtico' column: {missing_oil}")

# Handle missing values in the 'dcoilwtico' column by interpolation
oil_data['dcoilwtico'] = oil_data['dcoilwtico'].interpolate(method='linear')

# Handle any remaining missing values (e.g., at the start or end) with forward/backward fill
oil_data['dcoilwtico'].fillna(method='ffill', inplace=True)  # Forward fill
oil_data['dcoilwtico'].fillna(method='bfill', inplace=True)  # Backward fill (if needed)

# Verify no missing values remain
missing_oil_after = oil_data['dcoilwtico'].isna().sum()
#print(f"Number of missing values in 'dcoilwtico' column after handling all missing values: {missing_oil_after}")

#display(holiday_data)
# Drop 'locale_name' and 'description' columns from holiday_data
holiday_data.drop(columns=['locale_name', 'description'], inplace=True, errors='ignore')

# Display the updated holiday_data to verify
#print("Updated holiday_data:")
#display(holiday_data)

# Merge merged_data with holiday_data on the 'date' column
merged_data_with_holiday = pd.merge(merged_data, holiday_data, on='date', how='left')

# Display the merged dataset
#print("Merged Data with Holiday Data:")
#display(merged_data_with_holiday)

# Merge merged_data_with_holiday with oil_data on the 'date' column
final_merged_data = pd.merge(merged_data_with_holiday, oil_data, on='date', how='left')

# Display the merged dataset
#print("Final Merged Data (with Oil Data):")
#display(final_merged_data)

# Select categorical columns for one-hot encoding, including 'family'
categorical_columns = ['type', 'locale', 'transferred', 'family'] 

# Perform one-hot encoding
final_encoded_data = pd.get_dummies(final_merged_data, columns=categorical_columns, prefix=categorical_columns, dtype=int)

# Display the encoded dataset
#print("Final Encoded Data:")
#display(final_encoded_data)

# Check for NaN values in the dataset
missing_values = final_encoded_data.isna().sum()

# Display only columns with missing values
missing_values = missing_values[missing_values > 0]

if missing_values.empty:
    print("No missing values in the dataset.")
else:
    print("Columns with missing values:")
    print(missing_values)

# Interpolate missing values in the 'dcoilwtico' column using linear interpolation
final_encoded_data['dcoilwtico'] = final_encoded_data['dcoilwtico'].interpolate(method='linear')

# Fill any remaining missing values at the beginning or end using forward and backward fill
final_encoded_data['dcoilwtico'].fillna(method='ffill', inplace=True)  # Forward fill
final_encoded_data['dcoilwtico'].fillna(method='bfill', inplace=True)  # Backward fill (if needed)

# Verify no missing values remain in 'dcoilwtico'
missing_dcoilwtico = final_encoded_data['dcoilwtico'].isna().sum()
#print(f"Remaining missing values in 'dcoilwtico': {missing_dcoilwtico}")

#print(final_encoded_data.isna().sum())

# Check for duplicate IDs before removal
duplicates_before = final_encoded_data['id'].duplicated().sum()
#print(f"Number of duplicate IDs before handling: {duplicates_before}")

# Remove duplicate rows based on the 'id' column, keeping the first occurrence
final_encoded_data = final_encoded_data[~final_encoded_data['id'].duplicated(keep='first')]

# Check for duplicate IDs after removal
duplicates_after = final_encoded_data['id'].duplicated().sum()
#print(f"Number of duplicate IDs after handling: {duplicates_after}")

# Display the decrease in duplicates
duplicates_removed = duplicates_before - duplicates_after
#print(f"Number of duplicates removed: {duplicates_removed}")

# Encode date features
final_encoded_data['year'] = pd.to_datetime(final_encoded_data['date']).dt.year
final_encoded_data['month'] = pd.to_datetime(final_encoded_data['date']).dt.month
final_encoded_data['day'] = pd.to_datetime(final_encoded_data['date']).dt.day
final_encoded_data['day_of_week'] = pd.to_datetime(final_encoded_data['date']).dt.dayofweek  # 0=Monday, 6=Sunday
final_encoded_data['is_weekend'] = final_encoded_data['day_of_week'].isin([5, 6]).astype(int)  # 1 if Saturday or Sunday

# Drop the original 'date' column
final_encoded_data.drop(columns=['date'], inplace=True)

# Display the updated dataset
print("Updated DataFrame after encoding and dropping 'date':")
display(final_encoded_data)







# Separate training and test datasets
train_data = final_encoded_data[final_encoded_data['sales'].notna()]  # Rows with non-NaN sales values
test_data = final_encoded_data[final_encoded_data['sales'].isna()]    # Rows with NaN sales values


# Define features and target for training
X = train_data.drop(columns=['sales', 'id'])  # Features
y = train_data['sales']  # Target

# Define features for testing
X_test = test_data.drop(columns=['sales', 'id'])

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = XGBRegressor(objective='reg:squarederror', random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = model.predict(X_val)

# Evaluate the model using MAE
mae = mean_absolute_error(y_val, y_val_pred)
print(f"Validation MAE: {mae}")

# Predict sales on the test dataset
test_data['sales'] = model.predict(X_test)

# Prepare the submission file
submission = test_data[['id', 'sales']].sort_values(by='id').reset_index(drop=True)
submission.to_csv('/kaggle/working/submission.csv', index=False)

# Display the first few rows of the submission file
print("Submission file preview:")
display(submission)


/tmp/ipykernel_17/934618069.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  oil_data['dcoilwtico'].fillna(method='ffill', inplace=True)  # Forward fill
/tmp/ipykernel_17/934618069.py:37: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  oil_data['dcoilwtico'].fillna(method='ffill', inplace=True)  # Forward fill
/tmp/ipykernel_17/934618069.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained 

Columns with missing values:
sales          28512
dcoilwtico    885654
dtype: int64


/tmp/ipykernel_17/934618069.py:92: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_encoded_data['dcoilwtico'].fillna(method='ffill', inplace=True)  # Forward fill
/tmp/ipykernel_17/934618069.py:92: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_encoded_data['dcoilwtico'].fillna(method='ffill', inplace=True)  # Forward fill
/tmp/ipykernel_17/934618069.py:93: FutureWarning: A value is trying to be set on a copy of a DataFrame or Ser

Updated DataFrame after encoding and dropping 'date':


,id,store_nbr,sales,onpromotion,dcoilwtico,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,...,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD,year,month,day,day_of_week,is_weekend
0,0,1,0.0,0,93.14,0,0,0,1,0,...,0,0,0,0,0,2013,1,1,1,0
1,1,1,0.0,0,93.14,0,0,0,1,0,...,0,0,0,0,0,2013,1,1,1,0
2,2,1,0.0,0,93.14,0,0,0,1,0,...,0,0,0,0,0,2013,1,1,1,0
3,3,1,0.0,0,93.14,0,0,0,1,0,...,0,0,0,0,0,2013,1,1,1,0
4,4,1,0.0,0,93.14,0,0,0,1,0,...,0,0,0,0,0,2013,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082855,3029395,9,NaN,1,47.26,0,0,0,0,0,...,1,0,0,0,0,2017,8,31,3,0
3082856,3029396,9,NaN,0,47.26,0,0,0,0,0,...,0,1,0,0,0,2017,8,31,3,0
3082857,3029397,9,NaN,1,47.26,0,0,0,0,0,...,0,0,1,0,0,2017,8,31,3,0
3082858,3029398,9,NaN,9,47.26,0,0,0,0,0,...,0,0,0,1,0,2017,8,31,3,0


Validation MAE: 102.50700245111082
Submission file preview:


/tmp/ipykernel_17/934618069.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['sales'] = model.predict(X_test)


,id,sales
0,3000888,-14.345853
1,3000889,-9.103026
2,3000890,-37.323532
3,3000891,2605.227539
4,3000892,-22.602394
...,...,...
28507,3029395,376.531128
28508,3029396,83.442253
28509,3029397,1211.333740
28510,3029398,163.126221
